데이터 전처리
- 비속어와 비속어가 아닌 문장을 분류

In [1]:
import pandas as pd

# TSV 파일 경로
tsv_file_path = 'korean_unsmile_dataset/unsmile_train_v1.0.tsv'

# TSV 파일 열기
df = pd.read_csv(tsv_file_path, sep='\t')

# 데이터프레임 출력
df


,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15000,께롱께롱께롱!!!,0,0,0,0,0,0,0,0,0,1,0
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0,0,0,0,0,1,0,0,0,0,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,0,0,1,0,0,0,0,0,0,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0,0,0,0,0,0,0,0,1,0,0


In [2]:
df2 = pd.concat([df['문장'], df['clean']], axis=1)

In [3]:
df2

,문장,clean
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...
15000,께롱께롱께롱!!!,1
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0


In [4]:
df2.to_csv('clean_or_dirty_words.csv', encoding='utf-8')

Pretrained BERT 사용

In [6]:
!pip install transformers

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------- ------------------------------ 30.7/134.8 kB 1.4 MB/s eta 0:00:01
     -------------------------------------  133.1/134.8 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.8 MB 10.5 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/8.8 MB 11.1 MB/s eta 0:00:01
   ------ --------------------------------- 1.5/8

In [4]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'sgunderscore/hatescore-korean-hate-speech'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        device = -1, # gpu: 0
        return_all_scores = True,
        function_to_apply = 'sigmoid')

c:\Users\bluecom014\miniconda3\envs\final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\bluecom014\miniconda3\envs\final\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
#text = "야 이 병신같은 새끼야. 대가리를 왜 달고 다님?"
text = '안녕하세요'
label_and_score = pipe(text)[0]

label_and_score = sorted(label_and_score, key=lambda x:-x['score'])

if label_and_score[0]['label'] == 'None':
  print('정상 댓글')

else:
  print('비속어 감지')
  print(f'{label_and_score[0]["label"]} 비하 글이므로 필터링 필요.')

정상 댓글


Pretrained 모델 정확도 측정

In [29]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

labels = df2['clean'].tolist()
predictions = []
for data in tqdm(df2['문장'].tolist()):
    result = pipe(data)[0]
    result = sorted(result, key=lambda x:-x['score'])
    predict_label = 1 if result[0]['label'] == 'None' else 0
    predictions.append(predict_label)

accuracy = accuracy_score(labels, predictions)
accuracy

  0%|          | 0/15005 [00:00<?, ?it/s]

100%|██████████| 15005/15005 [16:54<00:00, 14.79it/s]


0.852315894701766

0 비속어, 민감정보 둘 다 없음  false, false  
1 비속어만 있음  true false  
2 민감정보만 있음  false true  
3 비속어, 민감정보 둘 다 있음  true true  

In [8]:
import re

# sample_text = """
# 개인 정보 보호를 위해 전화번호 010-1234-5678와 이메일 test@example.com,
# 그리고 주민등록번호 920101-1234567을 마스킹합니다.
# """

sample_text = "010-1234-5678 이 번호로 연락오면 받아라. 안 받으면 죽여버린다 개새끼야."

def pattern_match(text):
    email_pattern = r'\b([A-Za-z0-9._%+-]{2})([A-Za-z0-9._%+-]+)@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_pattern = r'(\d{3})-(\d{4})-(\d{4})'
    resident_id_pattern = r'(\d{6})-?([1-4])(\d{6})'

    matches_email = re.findall(email_pattern, text)
    matches_phone = re.findall(phone_pattern, text)
    matches_id = re.findall(resident_id_pattern, text)
    
    return matches_email or matches_phone or matches_id

# if pattern_match(sample_text):
#     print("패턴 적용")
# else:
#     print("패턴 적용 x")

def bad_words_find(text):
    label_and_score = pipe(text)[0]
    label_and_score = sorted(label_and_score, key=lambda x:-x['score'])
    return False if label_and_score[0]['label'] == 'None' else True

# bad_words_find(sample_text)

if bad_words_find(sample_text):
    label = 3 if pattern_match(sample_text) else 1
else:
    label = 2 if pattern_match(sample_text) else 0

label

3

In [17]:
[chr(i) for i in range(ord('ㄱ'), ord('ㅎ')+1)]

['ㄱ',
 'ㄲ',
 'ㄳ',
 'ㄴ',
 'ㄵ',
 'ㄶ',
 'ㄷ',
 'ㄸ',
 'ㄹ',
 'ㄺ',
 'ㄻ',
 'ㄼ',
 'ㄽ',
 'ㄾ',
 'ㄿ',
 'ㅀ',
 'ㅁ',
 'ㅂ',
 'ㅃ',
 'ㅄ',
 'ㅅ',
 'ㅆ',
 'ㅇ',
 'ㅈ',
 'ㅉ',
 'ㅊ',
 'ㅋ',
 'ㅌ',
 'ㅍ',
 'ㅎ']

In [9]:
# https://frhyme.github.io/python/python_korean_englished/
# 문장을 자음, 모음 나누기

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def korean_to_be_englished(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst
    
korean_to_be_englished("취뽀하고 싶다. 나 좀 데려가줘~~")

[['ㅊ', 'ㅟ', ' '],
 ['ㅃ', 'ㅗ', ' '],
 ['ㅎ', 'ㅏ', ' '],
 ['ㄱ', 'ㅗ', ' '],
 [' '],
 ['ㅅ', 'ㅣ', 'ㅍ'],
 ['ㄷ', 'ㅏ', ' '],
 ['.'],
 [' '],
 ['ㄴ', 'ㅏ', ' '],
 [' '],
 ['ㅈ', 'ㅗ', 'ㅁ'],
 [' '],
 ['ㄷ', 'ㅔ', ' '],
 ['ㄹ', 'ㅕ', ' '],
 ['ㄱ', 'ㅏ', ' '],
 ['ㅈ', 'ㅝ', ' '],
 ['~'],
 ['~']]

https://scienceon.kisti.re.kr/commons/util/originalView.do?cn=CFKO201924664108493&oCn=NPAP13263940&dbt=CFKO&journal=NPRO00383455  


In [10]:
# 자음, 모음 카데고리 사전 만들기
NUMBER_LIST = list(map(str, [i for i in range(10)]))
ALPHABET_LIST = [chr(i) for i in range(ord('a'), ord('z')+1)]
SPECIAL_WORD_LIST = ['!', '"', '#', '$', '%', '&', '\'', '?','@', '*', '+',',','-','.', '/', '~', ':', '^', ' ']
ONEWORD_LIST = ['ㄳ', 'ㄵ', 'ㅄ', 'ㄺ']

word_vector_list = CHOSUNG_LIST+JUNGSUNG_LIST+JONGSUNG_LIST+NUMBER_LIST+ALPHABET_LIST+SPECIAL_WORD_LIST+ONEWORD_LIST
print(word_vector_list)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '?', '@', '*', '+', ',', '-', '.', '/', '~', ':', '^', ' ', 'ㄳ', 'ㄵ', 'ㅄ', 'ㄺ']


In [11]:
# 카데고리 사전의 각 데이터와 인덱스를 딕셔너리로 추출
# 자음일 때 예를 들어 'ㄱ': [0, 41]은 ㄱ이 초성으로 오면 0, 종성으로 오면 41
# ' '(공백) 이면 ' ': [40, 122]. 40은 글자에 종성이 없을 때. 122는 단어와 단어 사이 띄어쓰기

word_vector_idx_dict = {}

for idx, data in enumerate(word_vector_list):
    if not data in word_vector_idx_dict:
        word_vector_idx_dict[data] = [] 
    word_vector_idx_dict[data].append(idx)
print(word_vector_idx_dict)

{'ㄱ': [0, 41], 'ㄲ': [1, 42], 'ㄴ': [2, 44], 'ㄷ': [3, 47], 'ㄸ': [4], 'ㄹ': [5, 48], 'ㅁ': [6, 56], 'ㅂ': [7, 57], 'ㅃ': [8], 'ㅅ': [9, 59], 'ㅆ': [10, 60], 'ㅇ': [11, 61], 'ㅈ': [12, 62], 'ㅉ': [13], 'ㅊ': [14, 63], 'ㅋ': [15, 64], 'ㅌ': [16, 65], 'ㅍ': [17, 66], 'ㅎ': [18, 67], 'ㅏ': [19], 'ㅐ': [20], 'ㅑ': [21], 'ㅒ': [22], 'ㅓ': [23], 'ㅔ': [24], 'ㅕ': [25], 'ㅖ': [26], 'ㅗ': [27], 'ㅘ': [28], 'ㅙ': [29], 'ㅚ': [30], 'ㅛ': [31], 'ㅜ': [32], 'ㅝ': [33], 'ㅞ': [34], 'ㅟ': [35], 'ㅠ': [36], 'ㅡ': [37], 'ㅢ': [38], 'ㅣ': [39], ' ': [40, 122], 'ㄳ': [43, 123], 'ㄵ': [45, 124], 'ㄶ': [46], 'ㄺ': [49, 126], 'ㄻ': [50], 'ㄼ': [51], 'ㄽ': [52], 'ㄾ': [53], 'ㄿ': [54], 'ㅀ': [55], 'ㅄ': [58, 125], '0': [68], '1': [69], '2': [70], '3': [71], '4': [72], '5': [73], '6': [74], '7': [75], '8': [76], '9': [77], 'a': [78], 'b': [79], 'c': [80], 'd': [81], 'e': [82], 'f': [83], 'g': [84], 'h': [85], 'i': [86], 'j': [87], 'k': [88], 'l': [89], 'm': [90], 'n': [91], 'o': [92], 'p': [93], 'q': [94], 'r': [95], 's': [96], 't': [97], 'u': [98], 'v': [9

In [12]:
# 자음 중 초성과 종성에서 종성에만 있는 데이터를 따로 추출

JONGSUNG_ONLY = sorted(list(set(JONGSUNG_LIST)-set(CHOSUNG_LIST)))[1:]
JONGSUNG_ONLY

['ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']

In [13]:
# 입력한 문장의 자음, 모음 벡터화

def make_input_vector(text):
    input_vector = [[0 for _ in range(127)] for _ in range(200)]
    text = text.lower()  # 영어는 소문자로 바꿈

    consonants_and_vowels = korean_to_be_englished(text)

    if len(sum(consonants_and_vowels,[]))<200:  # 입력 문자의 벡터는 200자 이내로 제한
        cnt = 0  # input_vector의 index
        for words in consonants_and_vowels:
            if len(words) == 3:  # 한글일 때 (초성, 중성, 종성)
                for idx in range(3):
                    if idx == 0 or idx == 1:  # 초성, 중성
                        input_vector[cnt][word_vector_idx_dict[words[idx]][0]]=1
                    
                    else:  # 종성
                        if words[idx] == ' ' or words[idx] in JONGSUNG_ONLY:  # 종성이 없을 때, 자음 중 종성만 있을 때
                            input_vector[cnt][word_vector_idx_dict[words[idx]][0]]=1
                        else:  # 자음 중 초성과 종성 둘 다 있는 경우, 종성
                            input_vector[cnt][word_vector_idx_dict[words[idx]][1]]=1
                    cnt += 1
                    
            else:  # 한글이 아닌 경우 (특수문자, 영어)
                if words[0] not in word_vector_list:  # word_vector_list에 없는 특수문자는 제외
                    pass
                elif words[0] == ' ':  # 띄어쓰기인 경우
                    input_vector[cnt][word_vector_idx_dict[words[0]][1]]=1
                else:  # 특수문자, 영어인 경우
                    input_vector[cnt][word_vector_idx_dict[words[0]][0]]=1
                cnt += 1

        return input_vector
    
    else:
        return

In [14]:
# tsv의 문장들을 one hot vector로 변환

import pandas as pd
import numpy as np

pickle_data = pd.DataFrame(columns=['sentence', 'onehot_vector', 'clean'])
for data in df2['문장']:
    data_vector = np.array(make_input_vector(data))
    if data_vector.shape == (200, 127):
        target = df2.loc[df2['문장']==data].values[0][1]
        pickle_data = pd.concat([pickle_data, pd.DataFrame({'sentence':[data], 'onehot_vector':[data_vector], 'clean':[target]})], ignore_index=True)
    

In [15]:
pickle_data

,sentence,onehot_vector,clean
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,"[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,예수 십새끼 개새끼 창녀아들 애비실종 가정교육 못받은 무뇌충 허언증 정신병자 사기꾼...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",0
...,...,...,...
13433,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...",0
13434,께롱께롱께롱!!!,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
13435,갱상도가아니라 홍어지 개좃같은 홍어년들,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
13436,말레이시아랑 인도네시아 여자 존나 못생겼던데,"[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",0


In [16]:
# pickle 파일로 저장 (용량이 커서 gzip 사용)
import pickle
import gzip

with gzip.open('hs.pickle', 'wb') as f:
    pickle.dump(pickle_data, f)

In [ ]:
# pickle 파일 읽기
import pickle
import gzip

with gzip.open('hs.pickle', 'rb') as f:
    data = pickle.load(f)
